In [1]:
#python 3.6
%%writefile requirements.txt
Keras==2.1.2
tensorflow==1.4.1


Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached Keras-2.1.2-py2.py3-none-any.whl (304 kB)
ERROR: Could not find a version that satisfies the requirement tensorflow==1.4.1 (from versions: 1.13.1, 1.13.2, 1.14.0, 1.15.0, 1.15.2, 1.15.3, 1.15.4, 1.15.5, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.1.0, 2.1.1, 2.1.2, 2.1.3, 2.1.4, 2.2.0, 2.2.1, 2.2.2, 2.2.3, 2.3.0, 2.3.1, 2.3.2, 2.3.3, 2.3.4, 2.4.0, 2.4.1, 2.4.2, 2.4.3, 2.4.4, 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.0+zzzcolab20220506153740, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.4+zzzcolab20220516125453, 2.6.5, 2.6.5+zzzcolab20220523104206, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.0+zzzcolab20220506150900, 2.7.1, 2.7.2, 2.7.2+zzzcolab20220516114640, 2.7.3, 2.7.3+zzzcolab20220523111007, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.0+zzzcolab20220506162203, 2.8.1, 2.8.1+zzzcolab20220516111314, 2.8.1+zzzcolab20220518083849, 2.8.2, 2.8.2+zzzcolab20220523105045, 2.8.2+zzzcolab20220

# Importando e definindo funções necessárias

In [6]:
import numpy as np
import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape, Concatenate, Flatten
from tensorflow.keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from neural_collaborative_filtering.Dataset import Dataset
from neural_collaborative_filtering.evaluate import evaluate_model
from time import time
import multiprocessing as mp
import sys
import math
import argparse




ModuleNotFoundError: ignored

get_model: gera os embeddings, faz o flatten, faz o element-wise e por fim faz a camada de ativação

In [10]:
def init_normal(shape, name=None):
    return initializers.normal()

def get_model(num_users, num_items, latent_dim, regs=[0,0]):
    # Input variables
    user_input = Input(shape=(1,), dtype='int32', name = 'user_input')
    item_input = Input(shape=(1,), dtype='int32', name = 'item_input')

    
    #primeiro gera os embeddings
    MF_Embedding_User = Embedding(input_dim = num_users, output_dim = latent_dim, name = 'user_embedding', embeddings_initializer = 'random_normal', embeddings_regularizer = l2(regs[0]), input_length=1)
    MF_Embedding_Item = Embedding(input_dim = num_items, output_dim = latent_dim, name = 'item_embedding', embeddings_initializer = 'random_normal', embeddings_regularizer = l2(regs[1]), input_length=1)   


    # Crucial to flatten an embedding vector!
    user_latent = Flatten()(MF_Embedding_User(user_input))
    item_latent = Flatten()(MF_Embedding_Item(item_input))
    
    # Element-wise product of user and item embeddings 
    predict_vector = merge([user_latent, item_latent], mode = 'mul')
    
    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', kernel_initializer='lecun_uniform', name = 'prediction')(predict_vector)
    
    model = Model(inputs=[user_input, item_input], 
                outputs=prediction)

    return model

def get_train_instances(train, num_negatives):
    user_input, item_input, labels = [],[],[]
    num_users = train.shape[0]
    for (u, i) in train.keys():
        # positive instance
        user_input.append(u)
        item_input.append(i)
        labels.append(1)
        # negative instances
        for t in range(num_negatives):
            j = np.random.randint(num_items)
            while train.__contains__((u, j)):
                j = np.random.randint(num_items)
            user_input.append(u)
            item_input.append(j)
            labels.append(0)
    return user_input, item_input, labels

# Fixando valores para exemplificar

In [ ]:
num_factors = 8
regs = [0,0]
num_negatives = 4
learner = 'adam'
learning_rate = 0.001
epochs = 5
batch_size = 256
verbose = 1
dataset_name = 'ml-1m'
path = 'neural_collaborative_filtering/Data/'
topK = 10
evaluation_threads = 1 #mp.cpu_count()
out = 0

# Loading data

Negativos são usuários que selecionamos aleatoriamente mas que não iteragiram com os itens. Ou seja, aqueles com entrada zero na matriz.

In [ ]:
model_out_file = 'Pretrain/%s_GMF_%d_%d.h5' %(dataset_name, num_factors, time())

# Loading data
t1 = time()
dataset = Dataset(path + dataset_name)
train, testRatings, testNegatives = dataset.trainMatrix, dataset.testRatings, dataset.testNegatives
num_users, num_items = train.shape
print("Load data done [%.1f s]. #user=%d, #item=%d, #train=%d, #test=%d" 
      %(time()-t1, num_users, num_items, train.nnz, len(testRatings)))

Load data done [14.3 s]. #user=6040, #item=3706, #train=994169, #test=6040


# Visualizando estruturas

In [ ]:
train.toarray()

array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
testRatings

[[0, 25],
 [1, 133],
 [2, 207],
 [3, 208],
 [4, 222],
 [5, 396],
 [6, 74],
 [7, 91],
 [8, 514],
 [9, 659],
 [10, 820],
 [11, 829],
 [12, 844],
 [13, 105],
 [14, 519],
 [15, 218],
 [16, 246],
 [17, 228],
 [18, 770],
 [19, 641],
 [20, 1167],
 [21, 1195],
 [22, 934],
 [23, 603],
 [24, 872],
 [25, 352],
 [26, 839],
 [27, 1299],
 [28, 1458],
 [29, 1487],
 [30, 274],
 [31, 1521],
 [32, 459],
 [33, 91],
 [34, 1647],
 [35, 1071],
 [36, 808],
 [37, 244],
 [38, 237],
 [39, 1696],
 [40, 303],
 [41, 730],
 [42, 104],
 [43, 472],
 [44, 1772],
 [45, 1805],
 [46, 1624],
 [47, 360],
 [48, 968],
 [49, 1879],
 [50, 1595],
 [51, 788],
 [52, 245],
 [53, 1236],
 [54, 171],
 [55, 534],
 [56, 7],
 [57, 1719],
 [58, 170],
 [59, 488],
 [60, 2063],
 [61, 2119],
 [62, 743],
 [63, 876],
 [64, 693],
 [65, 41],
 [66, 82],
 [67, 19],
 [68, 485],
 [69, 237],
 [70, 317],
 [71, 18],
 [72, 1148],
 [73, 1646],
 [74, 939],
 [75, 267],
 [76, 445],
 [77, 1459],
 [78, 73],
 [79, 123],
 [80, 44],
 [81, 991],
 [82, 1904],
 [83

# Building Model

In [ ]:
model = get_model(num_users, num_items, num_factors, regs)
if learner.lower() == "adagrad": 
    model.compile(optimizer=Adagrad(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "rmsprop":
    model.compile(optimizer=RMSprop(lr=learning_rate), loss='binary_crossentropy')
elif learner.lower() == "adam":
    model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy')
else:
    model.compile(optimizer=SGD(lr=learning_rate), loss='binary_crossentropy')

/Users/dimassoares/opt/anaconda3/envs/mentorama3.6/lib/python3.6/site-packages/ipykernel_launcher.py:20: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


# Evaluate Model top K=10 model

In [ ]:
t1 = time()
(hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
hr, ndcg = np.array(hits).mean(), np.array(ndcgs).mean()

print('Init: HR = %.4f, NDCG = %.4f\t [%.1f s]' % (hr, ndcg, time() - t1))

Init: HR = 0.0955, NDCG = 0.0420	 [3.8 s]


# Train Model

In [ ]:
# NDCG: Normalized Discounted Cumulative Gain

In [ ]:
best_hr, best_ndcg, best_iter = hr, ndcg, -1
for epoch in range(epochs):
    t1 = time()
    # Generate training instances
    user_input, item_input, labels = get_train_instances(train, num_negatives)

    # Training
    hist = model.fit([np.array(user_input), np.array(item_input)], #input
                     np.array(labels), # labels 
                     batch_size=batch_size, nb_epoch=1, verbose=0, shuffle=True)
    t2 = time()

    # Evaluation
    if epoch %verbose == 0:
        (hits, ndcgs) = evaluate_model(model, testRatings, testNegatives, topK, evaluation_threads)
        hr, ndcg, loss = np.array(hits).mean(), np.array(ndcgs).mean(), hist.history['loss'][0]
        print('Iteration %d [%.1f s]: HR = %.4f, NDCG = %.4f, loss = %.4f [%.1f s]' 
              % (epoch,  t2-t1, hr, ndcg, loss, time()-t2))
        if hr > best_hr:
            best_hr, best_ndcg, best_iter = hr, ndcg, epoch
            if out > 0:
                model.save_weights(model_out_file, overwrite=True)

print("End. Best Iteration %d:  HR = %.4f, NDCG = %.4f. " %(best_iter, best_hr, best_ndcg))
if out > 0:
    print("The best GMF model is saved to %s" %(model_out_file))

/Users/dimassoares/opt/anaconda3/envs/mentorama3.6/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  # Remove the CWD from sys.path while we load stuff.


Iteration 0 [49.7 s]: HR = 0.5232, NDCG = 0.2949, loss = 0.3591 [4.1 s]
Iteration 1 [45.8 s]: HR = 0.5911, NDCG = 0.3363, loss = 0.2963 [4.2 s]
Iteration 2 [40.1 s]: HR = 0.6093, NDCG = 0.3481, loss = 0.2817 [3.6 s]
Iteration 3 [39.0 s]: HR = 0.6195, NDCG = 0.3553, loss = 0.2772 [3.4 s]
Iteration 4 [38.3 s]: HR = 0.6250, NDCG = 0.3603, loss = 0.2749 [3.4 s]
End. Best Iteration 4:  HR = 0.6250, NDCG = 0.3603. 


In [ ]:
user_input, item_input, labels = get_train_instances(train, num_negatives)

In [ ]:
np.array(user_input).shape

(4970845,)